# ROI Sanity Check
Checks if stored matched cells match fresh calculated.
This code could also be used to recalculate the clustering with different setttings

In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import vr2p
import gcsfs, pickle
from tqdm.notebook import tqdm
import yaml
from multiday_suite2p.transform import cluster_cell_masks

# Main Parameters

In [4]:
animals = ['A4','A5','A7','B2','B3','B4','B5','C1','D1','F1']

# cluster Settings
cluster_settings = """
criterion: "distance" # criterion used for clustering
threshold: 0.75 # Threshold used for clustering algorithm
min_sessions_perc: 50 #Exclude masks not present for this percentage of sessions (0-100).
min_perc: 50 #Create template based on pixels detected in X% of sesions.
step_sizes: [200,200] # clustering happens in these sizes blocks across the plane (for memory reasons)
bin_size: 50 # Look for masks around center+bin-size to avoid edge cases
min_distance: 20 # only masks with centers within this pixel radius of each other are considered for clustering.
min_size_non_overlap: 25 # minimum size of template mask in pixels.
  """
cluster_settings = yaml.safe_load(cluster_settings)

# Collect data.
def check_clustering(animal, cluster_settings):
    # get vr2p data
    data = vr2p.ExperimentData(f'gs://linear-2ac/Set A/Tyche-{animal}-SetA.zarr')
    im_size = data.images.registered[0]['mean_img'].shape
    # registration data.
    fs = gcsfs.GCSFileSystem(project='sprustonlab-meso')
    with fs.open(f'gs://linear-2ac/Registration_data_all_animals/register-Tyche-{animal}.pkl', 'rb') as handle:
        deforms, trans_images,trans_masks, trans_label_im = pickle.load(handle)
    # cluster matching data.
    with fs.open(f'gs://linear-2ac/Registration_data_all_animals/match-Tyche-{animal}.pkl', 'rb') as handle:
        matched_cells, matched_im, template_masks, template_im = pickle.load(handle)
    # store.
    stored = {'matched_cells':matched_cells, 'matched_im':matched_im, 'template_masks': template_masks, 'template_im':template_im}
    del matched_cells, matched_im, template_masks, template_im
    #  assert that number of sessions align.
    assert len(trans_masks)== len(data.images.registered) ,'Number of sessions in data and stored registration do not match'
    # cluster and create template masks.
    matched_cells, matched_im = cluster_cell_masks(trans_masks, im_size, cluster_settings)
    # test.
    check=True
    for i, cell in enumerate(matched_cells):
        centers = np.array([roi['med'] for roi in cell])
        stored_centers = np.array([roi['med'] for roi in matched_cells[i]])
        if not np.array_equal(centers, stored_centers):
            check= False
    print(animal, ' Equality: ',check)
    return check
for animal in animals:
    check_clustering(animal, cluster_settings)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [01:45<00:00, 21.08s/it]


A4  Equality:  True


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:35<00:00,  7.02s/it]


A5  Equality:  True


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [02:07<00:00, 25.48s/it]


...

C1  Equality:  True


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [02:34<00:00, 30.92s/it]


D1  Equality:  True


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:24<00:00,  6.21s/it]


F1  Equality:  True
